In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import jieba
from nltk.translate.bleu_score import sentence_bleu
from opencc import OpenCC

In [27]:

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import jieba
from nltk.translate.bleu_score import sentence_bleu
from opencc import OpenCC

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length):
        super(AttnDecoderRNN, self).__init__()
        # hidden_size也是embedding_dim 就是刚开始 每个单词的维度 
        self.hidden_size = hidden_size
        self.output_size = output_size 
        self.max_length = max_length
        self.embedding = nn.Embedding(self.output_size, self.hidden_size) # 跟上边的一样  embedding模型的定义 
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_2 = nn.Linear(self.max_length, 1)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size) #hidden_size=256

    def forward(self,decode_input,encoder_outputs,hidden):
        decode_input = decode_input.view(1, -1) # torch.Size([1, 256])
        
        input_1=decode_input.repeat(len(encoder_outputs),1)
        
        attn_weights =torch.tanh(self.attn(torch.cat([input_1,encoder_outputs], 1)))
        
        attn_weights=F.softmax(self.attn_2(attn_weights),dim=0).view(1,-1)
        # torch.cat： torch.Size([10, 512]) 把两个[10,256]的tensor 拼接 
        # attn_weights维度为  torch.Size([1, 10]),tensor([[0.0988, 0.0997, 0.0944, 0.1047, 0.1051, 0.0925, 0.1062, 0.1043, 0.0892,0.1052]], device='cuda:0', grad_fn=<ViewBackward>)
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))  # unsqueeze(0)在第一个维度上 增加一维 1 
        # torch.bmm 三维矩阵相乘  
        # attn_applied=torch.Size([1, 1, 256])=torch.bmm([1,1,10],[1,10,256]) 
        # print(attn_applied.size())
        
        output = torch.cat([decode_input , attn_applied[0]], 1)
        # output.size():torch.Size([1, 512])
        
        output = self.attn_combine(output).unsqueeze(0)
        # output.size():torch.Size([1, 1, 256])
        
        output = F.relu(output)
        #output.size():torch.Size([1, 1, 256])
        
        output, hidden = self.gru(output, hidden) # 输入端hidden:torch.Size([1, 1, 256])
        # output.size()：torch.Size([1, 1, 256])，输出端hidden.size()：torch.Size([1, 1, 256]) 
        
        output_last = F.log_softmax(self.out(output[0]), dim=1)
        #output.size()：torch.Size([1, 6756])
        
        return output_last,output, hidden, attn_weights
    
attentionRNN_decoder = AttnDecoderRNN(256,6756,10)

decode_input=torch.Tensor(1, 256)
encoder_outputs=torch.Tensor(10, 256)
hidden=torch.Tensor(1, 1, 256)
output_last,output, hidden, attn_weights=attentionRNN_decoder(decode_input,encoder_outputs,hidden)

print(output_last.size())
print(output.size())
print(hidden.size())
print(attn_weights.size())

torch.Size([1, 6756])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 10])


In [17]:
D = torch.Tensor(2,3)
print(D)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [28]:
import os
import shutil
def copyfile(path, outpath):
   dirfile = os.listdir(path)
   print((dirfile))
   for dirf in dirfile:
      i = 0
      for filename in dirf:
         i += 1
         file1 = str(int(filename) + i)
         newdir = path + '/' + dirf + '/' + file1 + '.jpg'
         # if not os.path.exists(outpath):
         #    os.makedirs(outpath)  #注意还有重命名的过程
         out = outpath + '/' + dirf + file1 + '.jpg'
         #shutil.copyfile(newdir, out)
         #i += 1
         print(('转换结束'))
if __name__ == "__main__":
   path = r'D:\b\b'
   outpath = 'D:/c'
   copyfile(path, outpath)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:\\b\\b'

In [ ]:
# 原文件夹
old_path = "G:/MP4"
# 查看原文件夹下所有的子文件夹
filenames = os.listdir(old_path)
# 新文件夹
target_path = "G:/MP5"
if not os.path.exists(target_path):
    os.mkdir(target_path)

for file in filenames:
    # 所有的子文件夹
    sonDir = "G:/MP4/" + file
    # 遍历子文件夹中所有的文件
    for root, dirs, files in os.walk(sonDir):
     # 如果文件夹中有文件
        if len(files) > 0:
            for f in files:
                newDir = sonDir + '/' + f
                # 将文件移动到新文件夹中
                shutil.move(newDir, target_path)
        else:
            print(sonDir + "文件夹是空的")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import time


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class neuralNetwork:
    """
    初始化
    """

    def __init__(self, innodes, hinodes, outnodes, learn):
        self.innodes = innodes
        self.hinodes = hinodes
        self.outnodes = outnodes
        self.wih = np.random.normal(0.0, pow(self.innodes, -0.5), (self.hinodes, self.innodes))  # (100,784)
        self.who = np.random.normal(0.0, pow(self.hinodes, -0.5), (self.outnodes, self.hinodes))  # (10,100)
        self.lr = learn
        self.activation_function = lambda x: scipy.special.expit(x)

    '''
     以图片形式展示数据集中的数字
    '''

    def show_num(self, image_array):
        image_array = image_array.reshape((28, 28))
        plt.imshow(image_array, cmap='Greys', interpolation='None')
        plt.show()

    '''
    数据集处理：由于使用sigmoid函数，将目标值设置到 0.01--1的范围内 并设置目标值集合
    '''

    def data_deal(self, data_path):
        data_train = open(data_path, 'r')
        data_list = data_train.readlines()
        data_train.close()
        train_list = []
        target_list = []
        num_list = []
        out_nodes = 10
        for value in data_list:
            train_value = value.split(',')
            train_list.append(np.asfarray(train_value[1:]) / 255.0 * 0.99 + 0.01)
            num_list.append(np.asfarray(train_value[0]))
            targets = np.zeros(out_nodes) + 0.01
            targets[int(train_value[0])] = 0.99
            target_list.append(targets)
        return np.array(target_list), np.array(train_list), np.array(num_list)

    '''
    训练函数：前向传播计算输出，同时反向传播更新权重矩阵
    使用的激活函数为sigmoid
    '''

    def train(self, train_set, target_set):
        input = np.array(train_set, ndmin=2).T
        target = np.array(target_set, ndmin=2).T

        hidden_input = np.dot(self.wih, input)
        hidden_output = self.activation_function(hidden_input)
        final_input = np.dot(self.who, hidden_output)
        final_output = self.activation_function(final_input)

        output_error = target - final_output
        hidden_error = np.dot(self.who.T, output_error)

        self.who += self.lr * np.dot((output_error * final_output * (1.0 - final_output)), np.transpose(hidden_output))
        self.wih += self.lr * np.dot((hidden_error * hidden_output * (1.0 - hidden_output)), np.transpose(input))

    '''
    使用train得到的权重矩阵计算最终输出值，用以和真实值进行对比，从而计算该神经网络的正确率
    '''

    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs


# 初始化
neural = neuralNetwork(784, 200, 10, 0.1)

'''
# 一开始使用100个数据的数据集进行训练，训练5次，虽然训练时的准确率挺高的 99%左右，但是在使用测试集进行测试时，准确率降到了60%
#推测为训练数据集中的数据不够随机，导致测试时准确度下降
target, train, num = neural.data_deal("dataSet/mnist_train_100.csv")
for record in range(5):
    score = []
    for i, j, n in zip(train, target, num):
        neural.train(i, j)
        outputs = neural.query(i)
        if n == np.argmax(outputs):
            score.append(n)
print(float(len(score) / len(num)))
'''
# 改用包含60000个数据的数据集进行训练，准确度提升至98% 且测试时的准确度在97%左右

target, train, num = neural.data_deal("dataSet/mnist_train_100.csv")
start_time = time.time()
for a in range(1000):
    score = []
    for i, j, n in zip(train, target, num):
        neural.train(i, j)
        outputs = neural.query(i)

        if n == np.argmax(outputs):
            score.append(n)
    if a % 100 == 0:
        print(float(len(score) / len(num)))
end_time = time.time()
print("消耗时间：", str(end_time - start_time))
# 由于测试集数据比较少，因此改用含有100个数据的训练集进行测试
# test_target, test_train, test_num = neural.data_deal("dataSet/mnist_test_10.csv")
# test_score = []
# num_list = []
# for o, p, q in zip(test_train, test_target, test_num):
#     outputs = neural.query(o)
#     num_list.append(np.argmax(outputs))
#     if q == np.argmax(outputs):
#         test_score.append(q)
#
# print(float(len(test_score) / len(test_num)))
# print(num_list)
# print(test_num)


In [11]:
a=torch.rand(1,256)
a=a.repeat(10,1)
print(a.size())
b=torch.rand(10,256)
attn = nn.Linear(512, 10)
weights = torch.tanh(attn(torch.cat([a, b],1)))
print(weights.size())
v=nn.Linear(10,1)
attention = F.softmax(v(weights),dim=0)
print(attention)

torch.Size([10, 256])
torch.Size([10, 10])
tensor([[0.0986],
        [0.1090],
        [0.0911],
        [0.0951],
        [0.1071],
        [0.1038],
        [0.1023],
        [0.0983],
        [0.0978],
        [0.0970]], grad_fn=<SoftmaxBackward>)


In [13]:
import torch
import torch.nn as nn
import numpy as np
class dot_attention(nn.Module):
    """ 点积注意力机制"""

    def __init__(self, attention_dropout=0.0):
        super(dot_attention, self).__init__()
        self.dropout = nn.Dropout(attention_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, scale=None, attn_mask=None):
        """
        前向传播
        :param q:
        :param k:
        :param v:
        :param scale:
        :param attn_mask:
        :return: 上下文张量和attention张量。
        """
        attention = torch.bmm(q, k.transpose(1, 2))
        if scale:
            attention = attention * scale        # 是否设置缩放
        if attn_mask:
            attention = attention.masked_fill(attn_mask, -np.inf)     # 给需要mask的地方设置一个负无穷。
        # 计算softmax
        attention = self.softmax(attention)
        # 添加dropout
        attention = self.dropout(attention)
        # 和v做点积。
        context = torch.bmm(attention, v)
        return context, attention


class MultiHeadAttention(nn.Module):
    """ 多头自注意力"""
    def __init__(self, model_dim=400, num_heads=4, dropout=0.0):
        super(MultiHeadAttention, self).__init__()

        self.dim_per_head = model_dim//num_heads   # 每个头的维度
        self.num_heads = num_heads
        self.linear_k = nn.Linear(model_dim, self.dim_per_head * num_heads)
        self.linear_v = nn.Linear(model_dim, self.dim_per_head * num_heads)
        self.linear_q = nn.Linear(model_dim, self.dim_per_head * num_heads)

        self.dot_product_attention = dot_attention(dropout)

        self.linear_final = nn.Linear(model_dim, model_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(model_dim)         # LayerNorm 归一化。

    def forward(self, key, value, query, attn_mask=None):
        # 残差连接
        residual = query

        dim_per_head = self.dim_per_head
        num_heads = self.num_heads
        batch_size = key.size(0)

        # 线性映射。
        key = self.linear_k(key)
        value = self.linear_v(value)
        query = self.linear_q(query)

        # 按照头进行分割
        key = key.view(batch_size * num_heads, -1, dim_per_head)
        value = value.view(batch_size * num_heads, -1, dim_per_head)
        query = query.view(batch_size * num_heads, -1, dim_per_head)

        if attn_mask:
            attn_mask = attn_mask.repeat(num_heads, 1, 1)

        # 缩放点击注意力机制
        scale = (key.size(-1) // num_heads) ** -0.5
        context, attention = self.dot_product_attention(query, key, value, scale, attn_mask)

        # 进行头合并 concat heads
        context = context.view(batch_size, -1, dim_per_head * num_heads)

        # 进行线性映射
        output = self.linear_final(context)

        # dropout
        output = self.dropout(output)

        # 添加残差层和正则化层。
        output = self.layer_norm(residual + output)

        return output, attention


if __name__ == '__main__':
    q = torch.ones((1, 17, 400))
    k = torch.ones((1, 17, 400))
    v = k
    mutil_head_attention = MultiHeadAttention()
    output, attention = mutil_head_attention(q, k, v)
    print("context:", output.size(), output)
    print("attention:", attention.size(), attention)






context: torch.Size([1, 17, 400]) tensor([[[0.6346, 0.0408, 2.2800,  ..., 0.5100, 0.5529, 0.4370],
         [0.6346, 0.0408, 2.2800,  ..., 0.5100, 0.5529, 0.4370],
         [0.6346, 0.0408, 2.2800,  ..., 0.5100, 0.5529, 0.4370],
         ...,
         [0.2948, 0.1023, 2.2115,  ..., 0.1425, 0.4066, 0.5442],
         [0.2948, 0.1023, 2.2115,  ..., 0.1425, 0.4066, 0.5442],
         [0.2948, 0.1023, 2.2115,  ..., 0.1425, 0.4066, 0.5442]]],
       grad_fn=<NativeLayerNormBackward>)
attention: torch.Size([4, 17, 17]) tensor([[[0.0326, 0.0494, 0.0941,  ..., 0.0941, 0.0658, 0.0326],
         [0.0721, 0.0342, 0.0467,  ..., 0.0467, 0.0789, 0.0721],
         [0.0332, 0.0739, 0.0409,  ..., 0.0409, 0.0937, 0.0332],
         ...,
         [0.0332, 0.0739, 0.0409,  ..., 0.0409, 0.0937, 0.0332],
         [0.0489, 0.0234, 0.0311,  ..., 0.0311, 0.1344, 0.0489],
         [0.0326, 0.0494, 0.0941,  ..., 0.0941, 0.0658, 0.0326]],

        [[0.0356, 0.0486, 0.0820,  ..., 0.0820, 0.0749, 0.0356],
         [0.